In [32]:
# Please execute/shift-return this cell everytime you run the notebook.  Don't edit it. 
%load_ext autoreload
%autoreload 2
from notebook import * 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


 # Memory hierarchy

## The memory hierarchy of my computer

In [33]:
# Hung-Wei's Desktop #1
! lscpu | grep "Model name"
! getconf -a | grep CACHE

Model name:                           13th Gen Intel(R) Core(TM) i7-13700
LEVEL1_ICACHE_SIZE                 65536
LEVEL1_ICACHE_ASSOC                
LEVEL1_ICACHE_LINESIZE             64
LEVEL1_DCACHE_SIZE                 32768
LEVEL1_DCACHE_ASSOC                8
LEVEL1_DCACHE_LINESIZE             64
LEVEL2_CACHE_SIZE                  4194304
LEVEL2_CACHE_ASSOC                 16
LEVEL2_CACHE_LINESIZE              64
LEVEL3_CACHE_SIZE                  31457280
LEVEL3_CACHE_ASSOC                 12
LEVEL3_CACHE_LINESIZE              64
LEVEL4_CACHE_SIZE                  
LEVEL4_CACHE_ASSOC                 
LEVEL4_CACHE_LINESIZE              


In [34]:
# Hung-Wei's Desktop #2
! ssh htseng@articuno "lscpu | grep 'Model name'; getconf -a | grep CACHE"

Model name:                           Intel(R) Core(TM) i7-14700K
LEVEL1_ICACHE_SIZE                 65536
LEVEL1_ICACHE_ASSOC                
LEVEL1_ICACHE_LINESIZE             64
LEVEL1_DCACHE_SIZE                 32768
LEVEL1_DCACHE_ASSOC                8
LEVEL1_DCACHE_LINESIZE             64
LEVEL2_CACHE_SIZE                  4194304
LEVEL2_CACHE_ASSOC                 16
LEVEL2_CACHE_LINESIZE              64
LEVEL3_CACHE_SIZE                  34603008
LEVEL3_CACHE_ASSOC                 11
LEVEL3_CACHE_LINESIZE              64
LEVEL4_CACHE_SIZE                  
LEVEL4_CACHE_ASSOC                 
LEVEL4_CACHE_LINESIZE              


In [35]:
# Your CSE142 Cluster
! cse142 run "lscpu | grep 'Model name'; getconf -a | grep CACHE"

bash: line 0: cd: /nfshome/htseng/courses/CSE142/demo/memory: No such file or directory
Model name:                           12th Gen Intel(R) Core(TM) i3-12100F
bash: line 0: cd: /nfshome/htseng/courses/CSE142/demo/memory: No such file or directory
LEVEL1_ICACHE_SIZE                 32768
LEVEL1_ICACHE_ASSOC                8
LEVEL1_ICACHE_LINESIZE             64
LEVEL1_DCACHE_SIZE                 49152
LEVEL1_DCACHE_ASSOC                12
LEVEL1_DCACHE_LINESIZE             64
LEVEL2_CACHE_SIZE                  1310720
LEVEL2_CACHE_ASSOC                 10
LEVEL2_CACHE_LINESIZE              64
LEVEL3_CACHE_SIZE                  12582912
LEVEL3_CACHE_ASSOC                 12
LEVEL3_CACHE_LINESIZE              64
LEVEL4_CACHE_SIZE                  0
LEVEL4_CACHE_ASSOC                 0
LEVEL4_CACHE_LINESIZE              0


In [36]:
# Jetson Nano (NV Tegra X1)
! ssh htseng@nano-2 "lscpu | grep 'Model name'; getconf -a | grep CACHE"

Model name:          Cortex-A57
LEVEL1_ICACHE_SIZE                 0
LEVEL1_ICACHE_ASSOC                0
LEVEL1_ICACHE_LINESIZE             64
LEVEL1_DCACHE_SIZE                 0
LEVEL1_DCACHE_ASSOC                0
LEVEL1_DCACHE_LINESIZE             64
LEVEL2_CACHE_SIZE                  0
LEVEL2_CACHE_ASSOC                 0
LEVEL2_CACHE_LINESIZE              0
LEVEL3_CACHE_SIZE                  0
LEVEL3_CACHE_ASSOC                 0
LEVEL3_CACHE_LINESIZE              0
LEVEL4_CACHE_SIZE                  0
LEVEL4_CACHE_ASSOC                 0
LEVEL4_CACHE_LINESIZE              0


In [37]:
compare([do_render_code("./madd/madd_A.c", lang="c++", show=["//START", "//END"]),do_render_code("./madd/madd_B.c", lang="c++", show=["//START", "//END"])])

## The "Locality" of my code

Take a look of matrix vector:

In [38]:
render_code("./mv/matvec.c", lang="c++", show=["//I_FIRST_START", "//I_FIRST_END"])

// ./mv/matvec.c:35-49 (15 lines)
  //I_FIRST_START
    for(i = 0; i < ARRAY_SIZE; i++)
    {
      result = 0.0;    
      for(j = 0; j < ARRAY_SIZE; j++)
      {
        result += a[i][j]*b[j];
        if(dump)
            printf("&a[%d][%d], %p\n&b[%d], %p\n", i, j, &a[i][j], j, &b[j]);
      }
      c[i] = result;
      if(dump)
          printf("&c[%d], %p\n", i, &c[i]);
    }
  //I_FIRST_END

In [39]:
! cd mv; make clean; make; ./matvec 16 1 1 >> mv_addresses.csv ; head -n 100 mv_addresses.csv

rm -f matvec
cc -DHAVE_LINUX_PERF_EVENT_H -g -O3  matvec.c -o matvec
&a[0][0], 0x55ac54a78330
&b[0], 0x55ac54a78c30
&a[0][1], 0x55ac54a78338
&b[1], 0x55ac54a78c38
&a[0][2], 0x55ac54a78340
&b[2], 0x55ac54a78c40
&a[0][3], 0x55ac54a78348
&b[3], 0x55ac54a78c48
&a[0][4], 0x55ac54a78350
&b[4], 0x55ac54a78c50
&a[0][5], 0x55ac54a78358
&b[5], 0x55ac54a78c58
&a[0][6], 0x55ac54a78360
&b[6], 0x55ac54a78c60
&a[0][7], 0x55ac54a78368
&b[7], 0x55ac54a78c68
&a[0][8], 0x55ac54a78370
&b[8], 0x55ac54a78c70
&a[0][9], 0x55ac54a78378
&b[9], 0x55ac54a78c78
&a[0][10], 0x55ac54a78380
&b[10], 0x55ac54a78c80
&a[0][11], 0x55ac54a78388
&b[11], 0x55ac54a78c88
&a[0][12], 0x55ac54a78390
&b[12], 0x55ac54a78c90
&a[0][13], 0x55ac54a78398
&b[13], 0x55ac54a78c98
&a[0][14], 0x55ac54a783a0
&b[14], 0x55ac54a78ca0
&a[0][15], 0x55ac54a783a8
&b[15], 0x55ac54a78ca8
&c[0], 0x55ac54a78cc0
&a[1][0], 0x55ac54a783c0
&b[0], 0x55ac54a78c30
&a[1][1], 0x55ac54a783c8
&b[1], 0x55ac54a78c38
&a[1][2], 0x55ac54a783d0
&b[2], 0x55ac54a78c40
&a[1

In [40]:
! cd mv; ./matvec 16 1 0 >> mv_addresses_j_first.csv ; head -n 100 mv_addresses_j_first.csv

&a[0][0], 0x55e60cd61330
&b[0], 0x55e60cd61c30
&a[1][0], 0x55e60cd613c0
&b[0], 0x55e60cd61c30
&a[2][0], 0x55e60cd61450
&b[0], 0x55e60cd61c30
&a[3][0], 0x55e60cd614e0
&b[0], 0x55e60cd61c30
&a[4][0], 0x55e60cd61570
&b[0], 0x55e60cd61c30
&a[5][0], 0x55e60cd61600
&b[0], 0x55e60cd61c30
&a[6][0], 0x55e60cd61690
&b[0], 0x55e60cd61c30
&a[7][0], 0x55e60cd61720
&b[0], 0x55e60cd61c30
&a[8][0], 0x55e60cd617b0
&b[0], 0x55e60cd61c30
&a[9][0], 0x55e60cd61840
&b[0], 0x55e60cd61c30
&a[10][0], 0x55e60cd618d0
&b[0], 0x55e60cd61c30
&a[11][0], 0x55e60cd61960
&b[0], 0x55e60cd61c30
&a[12][0], 0x55e60cd619f0
&b[0], 0x55e60cd61c30
&a[13][0], 0x55e60cd61a80
&b[0], 0x55e60cd61c30
&a[14][0], 0x55e60cd61b10
&b[0], 0x55e60cd61c30
&a[15][0], 0x55e60cd61ba0
&b[0], 0x55e60cd61c30
&c[16], 0x55e60cd61d40
&a[0][1], 0x55e60cd61338
&b[1], 0x55e60cd61c38
&a[1][1], 0x55e60cd613c8
&b[1], 0x55e60cd61c38
&a[2][1], 0x55e60cd61458
&b[1], 0x55e60cd61c38
&a[3][1], 0x55e60cd614e8
&b[1], 0x55e60cd61c38
&a[4][1], 0x55e60cd61578
&b[1],

## How blocks are stored in a cache?

How these address are stored "if" they're in a direct-mapped, 16B-sized blocks, 16-block cache?

In [41]:
! cd mv; make clean; make; ./matvec 16 1 

rm -f matvec
cc -DHAVE_LINUX_PERF_EVENT_H -g -O3  matvec.c -o matvec
&a[0][0], 0x5e43c58ab330
&b[0], 0x5e43c58abc30
&a[0][1], 0x5e43c58ab338
&b[1], 0x5e43c58abc38
&a[0][2], 0x5e43c58ab340
&b[2], 0x5e43c58abc40
&a[0][3], 0x5e43c58ab348
&b[3], 0x5e43c58abc48
&a[0][4], 0x5e43c58ab350
&b[4], 0x5e43c58abc50
&a[0][5], 0x5e43c58ab358
&b[5], 0x5e43c58abc58
&a[0][6], 0x5e43c58ab360
&b[6], 0x5e43c58abc60
&a[0][7], 0x5e43c58ab368
&b[7], 0x5e43c58abc68
&a[0][8], 0x5e43c58ab370
&b[8], 0x5e43c58abc70
&a[0][9], 0x5e43c58ab378
&b[9], 0x5e43c58abc78
&a[0][10], 0x5e43c58ab380
&b[10], 0x5e43c58abc80
&a[0][11], 0x5e43c58ab388
&b[11], 0x5e43c58abc88
&a[0][12], 0x5e43c58ab390
&b[12], 0x5e43c58abc90
&a[0][13], 0x5e43c58ab398
&b[13], 0x5e43c58abc98
&a[0][14], 0x5e43c58ab3a0
&b[14], 0x5e43c58abca0
&a[0][15], 0x5e43c58ab3a8
&b[15], 0x5e43c58abca8
&c[0], 0x5e43c58abcc0
&a[1][0], 0x5e43c58ab3c0
&b[0], 0x5e43c58abc30
&a[1][1], 0x5e43c58ab3c8
&b[1], 0x5e43c58abc38
&a[1][2], 0x5e43c58ab3d0
&b[2], 0x5e43c58abc40
&a[1

In [42]:
!echo "element,address"> addresses.csv; ./mv/matvec 16 1 >> addresses.csv
df = pd.read_csv("addresses.csv",skipfooter=1,engine='python')
df["address"] = df["address"].str.replace('0x','')
df["address"]=df[["address"]].apply(lambda x: x.astype(str).map(lambda x: int(x, base=16)))
# only show the first N addresses 
#N = 32
#df2 = df2.iloc[:N]
block_size = 16
offset_bits = int(math.log2(block_size))
number_of_blocks = 16
index_bits = int(math.log2(number_of_blocks))
df["tag"]=(df["address"].apply(lambda x: x >> (offset_bits+index_bits)))
df["tag"] = df["tag"].apply(lambda x: hex(x))
df["index"] = df["address"].apply(lambda x: hex((x>>offset_bits) % number_of_blocks))
df["address"] = df["address"].apply(lambda x: hex(x))
display_df_mono(df)

,element,address,tag,index
0,&a[0][0],0x574818c2c330,0x574818c2c3,0x3
1,&b[0],0x574818c2cc30,0x574818c2cc,0x3
2,&a[0][1],0x574818c2c338,0x574818c2c3,0x3
3,&b[1],0x574818c2cc38,0x574818c2cc,0x3
4,&a[0][2],0x574818c2c340,0x574818c2c3,0x4
5,&b[2],0x574818c2cc40,0x574818c2cc,0x4
6,&a[0][3],0x574818c2c348,0x574818c2c3,0x4
7,&b[3],0x574818c2cc48,0x574818c2cc,0x4
8,&a[0][4],0x574818c2c350,0x574818c2c3,0x5
9,&b[4],0x574818c2cc50,0x574818c2cc,0x5


What if we have a 2-way, 16-byte blocked, 16-block cache?

In [43]:
df = pd.read_csv("addresses.csv",skipfooter=1,engine='python')
df["address"] = df["address"].str.replace('0x','')
df["address"]=df[["address"]].apply(lambda x: x.astype(str).map(lambda x: int(x, base=16)))
# only show the first N addresses 
#N = 32
#df2 = df2.iloc[:N]
block_size = 16
offset_bits = int(math.log2(block_size))
number_of_blocks = 16
way_assoc=2
number_of_sets = number_of_blocks/way_assoc
index_bits = int(math.log2(number_of_sets))
df["tag"]=(df["address"].apply(lambda x: x >> (offset_bits+index_bits)))
df["tag"] = df["tag"].apply(lambda x: hex(x))
df["index"] = df["address"].apply(lambda x: hex((x>>offset_bits) % number_of_blocks))
df["address"] = df["address"].apply(lambda x: hex(x))
display_df_mono(df)

,element,address,tag,index
0,&a[0][0],0x574818c2c330,0xae90318586,0x3
1,&b[0],0x574818c2cc30,0xae90318598,0x3
2,&a[0][1],0x574818c2c338,0xae90318586,0x3
3,&b[1],0x574818c2cc38,0xae90318598,0x3
4,&a[0][2],0x574818c2c340,0xae90318586,0x4
5,&b[2],0x574818c2cc40,0xae90318598,0x4
6,&a[0][3],0x574818c2c348,0xae90318586,0x4
7,&b[3],0x574818c2cc48,0xae90318598,0x4
8,&a[0][4],0x574818c2c350,0xae90318586,0x5
9,&b[4],0x574818c2cc50,0xae90318598,0x5


## Cache performance of code on "real machines"

### NVIDIA Jetson Nano -- Tegra X1

In [44]:
render_code("4way_madd/madd.c", show=["//START","//END"])

// 4way_madd/madd.c:51-63 (13 lines)
  //START
    for(i = 0; i < RUN_ARRAY_SIZE; i++)
    {
        e[i] = (a[i] * b[i] + c[i])/d[i];
#ifdef DUMP
        fprintf(stderr, "a[%d], %p\n",i,&a[i]);
        fprintf(stderr, "b[%d], %p\n",i,&b[i]);
        fprintf(stderr, "c[%d], %p\n",i,&c[i]);
        fprintf(stderr, "d[%d], %p\n",i,&d[i]);
        fprintf(stderr, "e[%d], %p\n",i,&e[i]);
#endif
    }
  //END

Let's run it without the above loop code to figure the baseline memory accesses without running the loop on a Jetson nano.

#### Run without the 4-way matrix add loop code.

In [45]:
# Run it "without" the above code.
! ssh htseng@nano "lscpu; cd courses/CS203/demo/memory/4way_madd/; make clean madd_nano; valgrind --tool=cachegrind ./madd_nano 16384 0 "

Too much detail! Let's use grep to narrow down the outputs.

In [46]:
# Run it "without" the above code.
! ssh htseng@nano "cd courses/CS203/demo/memory/4way_madd/; valgrind --tool=cachegrind ./madd_nano 8192 0 >& nano_without_loop.perf; grep 'D   refs\|D1' nano_without_loop.perf"

Let's run it with the above loop code again and observe the changes in L1 cache misses/accesses

#### Run with the 4-way matrix add loop code.

In [47]:
! ssh htseng@nano "cd courses/CS203/demo/memory/4way_madd/;valgrind --tool=cachegrind ./madd_nano 8192 8192 >& nano_with_loop.perf; grep 'D   refs\|D1' nano_with_loop.perf"

In [48]:
# Let's do some math here
total_number_of_accesses_before_the_loop =    1527712
total_number_of_accesses_after_the_loop =  1548198
total_number_of_accesses_in_the_loop = total_number_of_accesses_after_the_loop-total_number_of_accesses_before_the_loop
total_number_of_misses_before_the_loop =  8543
total_number_of_misses_after_the_loop = 29036
total_number_of_misses_in_the_loop = total_number_of_misses_after_the_loop-total_number_of_misses_before_the_loop
miss_rate_of_the_loop = total_number_of_misses_in_the_loop/total_number_of_accesses_in_the_loop

print(f"access in the loop: %d misses in the loop %d miss_rate %lf" % (total_number_of_accesses_in_the_loop, total_number_of_misses_in_the_loop, miss_rate_of_the_loop))

access in the loop: 20486 misses in the loop 20493 miss_rate 1.000342


In [49]:
! cd 4way_madd; make madd_dump; cd ..; 
!echo "element,address"> addresses_madd.csv; 
!./4way_madd/madd_dump 8192 8192 2>> addresses_madd.csv
! head -n 101 addresses_madd.csv > addresses_digest.csv
df = pd.read_csv("addresses_digest.csv",skipfooter=1,engine='python')
df["address"] = df["address"].str.replace('0x','')
df["address"]=df[["address"]].apply(lambda x: x.astype(str).map(lambda x: int(x, base=16)))
# only show the first N addresses 
#N = 32
#df2 = df2.iloc[:N]
C = 32768
B = 64
A = 4
offset_bits = int(math.log2(B))
S = int(C/(B*A))
index_bits = int(math.log2(S))
df["tag"]=(df["address"].apply(lambda x: x >> (offset_bits+index_bits)))
df["tag"] = df["tag"].apply(lambda x: hex(x))
df["index"] = df["address"].apply(lambda x: hex((x>>offset_bits)%S))
df["address"] = df["address"].apply(lambda x: hex(x))
display_df_mono(df)

make: 'madd_dump' is up to date.
60.000000


,element,address,tag,index
0,a[0],0x77489761c000,0x3ba44bb0e,0x0
1,b[0],0x77489762c000,0x3ba44bb16,0x0
2,c[0],0x77489763c000,0x3ba44bb1e,0x0
3,d[0],0x77489764c000,0x3ba44bb26,0x0
4,e[0],0x77489765c000,0x3ba44bb2e,0x0
5,a[1],0x77489761c008,0x3ba44bb0e,0x0
6,b[1],0x77489762c008,0x3ba44bb16,0x0
7,c[1],0x77489763c008,0x3ba44bb1e,0x0
8,d[1],0x77489764c008,0x3ba44bb26,0x0
9,e[1],0x77489765c008,0x3ba44bb2e,0x0


### Intel Core i7 13700 -- 12-way L1, 64B-blocked, 48KB cache

Let's run it without the above loop code to figure the baseline memory accesses without running the loop on a Jetson nano.

Let's again dump, parse and simulation the address sequence.

In [50]:
! cd 4way_madd; make madd_dump; cd ..; 
!echo "element,address"> addresses_madd.csv; 
!./4way_madd/madd_dump 8192 8192 2>> addresses_madd.csv
! head -n 101 addresses_madd.csv > addresses_digest.csv
df = pd.read_csv("addresses_digest.csv",skipfooter=1,engine='python')
df["address"] = df["address"].str.replace('0x','')
df["address"]=df[["address"]].apply(lambda x: x.astype(str).map(lambda x: int(x, base=16)))
# only show the first N addresses 
#N = 32
#df2 = df2.iloc[:N]
C = 49152
B = 64
A = 12
offset_bits = int(math.log2(B))
S = int(C/(B*A))
index_bits = int(math.log2(S))
df["tag"]=(df["address"].apply(lambda x: x >> (offset_bits+index_bits)))
df["tag"] = df["tag"].apply(lambda x: hex(x))
df["index"] = df["address"].apply(lambda x: hex((x>>offset_bits) % S))
df["address"] = df["address"].apply(lambda x: hex(x))
display_df_mono(df)

make: 'madd_dump' is up to date.
60.000000


,element,address,tag,index
0,a[0],0x7d5bcef4e000,0x7d5bcef4e,0x0
1,b[0],0x7d5bcef5e000,0x7d5bcef5e,0x0
2,c[0],0x7d5bcef6e000,0x7d5bcef6e,0x0
3,d[0],0x7d5bcef7e000,0x7d5bcef7e,0x0
4,e[0],0x7d5bcef8e000,0x7d5bcef8e,0x0
5,a[1],0x7d5bcef4e008,0x7d5bcef4e,0x0
6,b[1],0x7d5bcef5e008,0x7d5bcef5e,0x0
7,c[1],0x7d5bcef6e008,0x7d5bcef6e,0x0
8,d[1],0x7d5bcef7e008,0x7d5bcef7e,0x0
9,e[1],0x7d5bcef8e008,0x7d5bcef8e,0x0


#### Run without the 4-way matrix add loop code.

In [51]:
# Run it "without" the above code.
! lscpu; cd ~/courses/CS203/demo/memory/4way_madd/; rm madd_intel; make madd_intel; valgrind --tool=cachegrind ./madd_intel 16384 0 >& intel_without_loop.perf; grep 'D   refs\|D1' intel_without_loop.perf

Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   24
  On-line CPU(s) list:    0-23
Vendor ID:                GenuineIntel
  Model name:             13th Gen Intel(R) Core(TM) i7-13700
    CPU family:           6
    Model:                183
    Thread(s) per core:   2
    Core(s) per socket:   16
    Socket(s):            1
    Stepping:             1
    CPU(s) scaling MHz:   31%
    CPU max MHz:          5200.0000
    CPU min MHz:          800.0000
    BogoMIPS:             4224.00
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush dts acpi mmx fxsr sse sse2 s
                          s ht tm pbe syscall nx pdpe1gb rdtscp lm constant_tsc 
                          art arch_perfmon pebs bts rep_good nopl xtopology nons
                          top_tsc c

Let's run it with the above loop code again and observe the changes in L1 cache misses/accesses

#### Run with the 4-way matrix add loop code.

In [52]:
# Run it "with" the above code.
! cd ~/courses/CS203/demo/memory/4way_madd/; valgrind --tool=cachegrind ./madd_intel 16384 16384  >& intel_with_loop.perf; grep 'D   refs\|D1' intel_with_loop.perf

In [53]:
# Let's do some math here
total_number_of_accesses_before_the_loop =2285911
total_number_of_accesses_after_the_loop = 2326877
total_number_of_accesses_in_the_loop = total_number_of_accesses_after_the_loop-total_number_of_accesses_before_the_loop
total_number_of_misses_before_the_loop = 13447
total_number_of_misses_after_the_loop = 23694
total_number_of_misses_in_the_loop = total_number_of_misses_after_the_loop-total_number_of_misses_before_the_loop
miss_rate_of_the_loop = total_number_of_misses_in_the_loop/total_number_of_accesses_in_the_loop

print(f"access in the loop: %d misses in the loop %d miss_rate %lf" % (total_number_of_accesses_in_the_loop, total_number_of_misses_in_the_loop, miss_rate_of_the_loop))

access in the loop: 40966 misses in the loop 10247 miss_rate 0.250134
